# Bootstrap Estimation and Method of Moments
## DA5400W - Foundations of Machine Learning
### Instructor: Dr. Arun B Ayyar
### IIT Madras

---

This notebook covers two important parameter estimation techniques:
1. **Method of Moments (MoM)**: A classical parametric approach
2. **Bootstrap Estimation**: A modern non-parametric resampling technique

Both methods are essential tools in statistical inference and machine learning.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import pandas as pd

# Set style for better visualizations
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Set random seed for reproducibility
np.random.seed(42)

print("Libraries imported successfully!")

## Part 1: Method of Moments (MoM)

### 1.1 Introduction

**Method of Moments** is one of the oldest parameter estimation techniques, dating back to Karl Pearson (1894).

**Key Idea:**
- Match sample moments (statistics computed from data) with theoretical moments (expectations from the distribution)
- Solve the resulting equations to estimate parameters

**Steps:**
1. Compute sample moments: $m_k = \frac{1}{n}\sum_{i=1}^{n} x_i^k$
2. Express theoretical moments in terms of parameters: $\mu_k = E[X^k]$
3. Set sample moments equal to theoretical moments: $m_k = \mu_k$
4. Solve for the parameters

**Advantages:**
- Simple and intuitive
- Easy to compute
- Works when MLE is difficult

**Disadvantages:**
- May not be as efficient as MLE
- Estimates may not always exist or be unique

### 1.2 Example: Exponential Distribution

**Problem:** Given data $x_1, x_2, \ldots, x_n$ from $\text{Exp}(\lambda)$, estimate $\lambda$.

**Exponential Distribution:**
- PDF: $f(x|\lambda) = \lambda e^{-\lambda x}$ for $x \geq 0$
- Mean: $E[X] = \frac{1}{\lambda}$

**Method of Moments:**
1. First sample moment: $m_1 = \bar{x} = \frac{1}{n}\sum_{i=1}^{n} x_i$
2. First theoretical moment: $\mu_1 = E[X] = \frac{1}{\lambda}$
3. Set equal: $\bar{x} = \frac{1}{\lambda}$
4. Solve: $\hat{\lambda}_{MoM} = \frac{1}{\bar{x}} = \frac{n}{\sum_{i=1}^{n} x_i}$

In [ ]:
# Example: Exponential Distribution - Method of Moments

# True parameter
lambda_true = 2.0

# Generate sample data
n = 100
data_exp = np.random.exponential(scale=1/lambda_true, size=n)

# Method of Moments estimator
lambda_mom = 1 / np.mean(data_exp)

print("Exponential Distribution - Method of Moments")
print("=" * 60)
print(f"True parameter: λ = {lambda_true}")
print(f"Sample size: n = {n}")
print(f"Sample mean: {np.mean(data_exp):.4f}")
print(f"\nMoM Estimate: λ̂ = {lambda_mom:.4f}")
print(f"Estimation error: {abs(lambda_mom - lambda_true):.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram with true and estimated distributions
axes[0].hist(data_exp, bins=30, density=True, alpha=0.7, color='skyblue', edgecolor='black', label='Sample data')
x_range = np.linspace(0, max(data_exp), 1000)
axes[0].plot(x_range, lambda_true * np.exp(-lambda_true * x_range), 
             'r-', linewidth=2, label=f'True: λ={lambda_true}')
axes[0].plot(x_range, lambda_mom * np.exp(-lambda_mom * x_range), 
             'g--', linewidth=2, label=f'MoM: λ̂={lambda_mom:.3f}')
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].set_title('Exponential Distribution: Data vs Estimated', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Convergence with sample size
sample_sizes = np.arange(10, 1000, 10)
estimates = []
for size in sample_sizes:
    sample = np.random.exponential(scale=1/lambda_true, size=size)
    estimates.append(1 / np.mean(sample))

axes[1].plot(sample_sizes, estimates, 'b-', alpha=0.6, linewidth=1.5, label='MoM estimates')
axes[1].axhline(y=lambda_true, color='r', linestyle='--', linewidth=2, label=f'True λ={lambda_true}')
axes[1].fill_between(sample_sizes, lambda_true - 0.5, lambda_true + 0.5, 
                      color='red', alpha=0.1, label='±0.5 band')
axes[1].set_xlabel('Sample Size (n)', fontsize=12)
axes[1].set_ylabel('Estimated λ', fontsize=12)
axes[1].set_title('Convergence of MoM Estimator', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 1.3 Example: Normal Distribution

**Problem:** Given data $x_1, x_2, \ldots, x_n$ from $N(\mu, \sigma^2)$, estimate $\mu$ and $\sigma^2$.

**Normal Distribution:**
- Mean: $E[X] = \mu$
- Variance: $\text{Var}(X) = E[X^2] - (E[X])^2 = \sigma^2$

**Method of Moments:**
1. First moment: $m_1 = \bar{x} = \mu$ → $\hat{\mu}_{MoM} = \bar{x}$
2. Second moment: $m_2 = \frac{1}{n}\sum x_i^2 = \sigma^2 + \mu^2$
3. Solve: $\hat{\sigma}^2_{MoM} = \frac{1}{n}\sum (x_i - \bar{x})^2$

In [ ]:
# Example: Normal Distribution - Method of Moments

# True parameters
mu_true = 50
sigma_true = 10

# Generate sample data
n = 200
data_normal = np.random.normal(loc=mu_true, scale=sigma_true, size=n)

# Method of Moments estimators
mu_mom = np.mean(data_normal)
sigma2_mom = np.mean((data_normal - mu_mom)**2)  # Biased estimator
sigma_mom = np.sqrt(sigma2_mom)

print("Normal Distribution - Method of Moments")
print("=" * 60)
print(f"True parameters: μ = {mu_true}, σ = {sigma_true}")
print(f"Sample size: n = {n}")
print(f"\nMoM Estimates:")
print(f"  μ̂ = {mu_mom:.4f} (error: {abs(mu_mom - mu_true):.4f})")
print(f"  σ̂ = {sigma_mom:.4f} (error: {abs(sigma_mom - sigma_true):.4f})")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Histogram with distributions
axes[0].hist(data_normal, bins=30, density=True, alpha=0.7, color='lightcoral', edgecolor='black', label='Sample data')
x_range = np.linspace(min(data_normal), max(data_normal), 1000)
axes[0].plot(x_range, stats.norm.pdf(x_range, mu_true, sigma_true), 
             'r-', linewidth=2, label=f'True: μ={mu_true}, σ={sigma_true}')
axes[0].plot(x_range, stats.norm.pdf(x_range, mu_mom, sigma_mom), 
             'g--', linewidth=2, label=f'MoM: μ̂={mu_mom:.1f}, σ̂={sigma_mom:.1f}')
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].set_title('Normal Distribution: Data vs Estimated', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Q-Q plot
stats.probplot(data_normal, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot: Checking Normality Assumption', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 1.4 Example: Bernoulli Distribution

**Problem:** Given binary data $x_1, x_2, \ldots, x_n$ from $\text{Bernoulli}(p)$, estimate $p$.

**Bernoulli Distribution:**
- PMF: $P(X=x) = p^x(1-p)^{1-x}$ for $x \in \{0, 1\}$
- Mean: $E[X] = p$

**Method of Moments:**
- Set $\bar{x} = p$
- $\hat{p}_{MoM} = \bar{x} = \frac{1}{n}\sum_{i=1}^{n} x_i$

This is also the Maximum Likelihood Estimator (MLE) for the Bernoulli distribution!

In [ ]:
# Example: Bernoulli Distribution - Method of Moments

# True parameter
p_true = 0.7

# Generate sample data
n = 100
data_bernoulli = np.random.binomial(1, p_true, size=n)

# Method of Moments estimator
p_mom = np.mean(data_bernoulli)

print("Bernoulli Distribution - Method of Moments")
print("=" * 60)
print(f"True parameter: p = {p_true}")
print(f"Sample size: n = {n}")
print(f"Number of successes: {np.sum(data_bernoulli)}")
print(f"Number of failures: {n - np.sum(data_bernoulli)}")
print(f"\nMoM Estimate: p̂ = {p_mom:.4f}")
print(f"Estimation error: {abs(p_mom - p_true):.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar plot of outcomes
outcomes, counts = np.unique(data_bernoulli, return_counts=True)
axes[0].bar(outcomes, counts, color=['coral', 'skyblue'], edgecolor='black', alpha=0.7)
axes[0].axhline(y=n*p_true, color='r', linestyle='--', linewidth=2, label=f'Expected successes: {n*p_true:.0f}')
axes[0].axhline(y=n*(1-p_true), color='b', linestyle='--', linewidth=2, label=f'Expected failures: {n*(1-p_true):.0f}')
axes[0].set_xlabel('Outcome', fontsize=12)
axes[0].set_ylabel('Count', fontsize=12)
axes[0].set_title('Bernoulli Trials: Observed vs Expected', fontsize=14, fontweight='bold')
axes[0].set_xticks([0, 1])
axes[0].set_xticklabels(['Failure (0)', 'Success (1)'])
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Convergence with sample size
sample_sizes = np.arange(10, 2000, 20)
estimates = []
for size in sample_sizes:
    sample = np.random.binomial(1, p_true, size=size)
    estimates.append(np.mean(sample))

axes[1].plot(sample_sizes, estimates, 'b-', alpha=0.6, linewidth=1.5, label='MoM estimates')
axes[1].axhline(y=p_true, color='r', linestyle='--', linewidth=2, label=f'True p={p_true}')
axes[1].fill_between(sample_sizes, p_true - 0.1, p_true + 0.1, 
                      color='red', alpha=0.1, label='±0.1 band')
axes[1].set_xlabel('Sample Size (n)', fontsize=12)
axes[1].set_ylabel('Estimated p', fontsize=12)
axes[1].set_title('Convergence of MoM Estimator', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Part 2: Bootstrap Estimation

### 2.1 Introduction to Bootstrap

**Bootstrap** is a powerful resampling technique introduced by Bradley Efron in 1979.

**Key Idea:**
- Treat the sample as a "population"
- Resample from it (with replacement) many times
- Compute the statistic of interest for each resample
- Use the distribution of these statistics to estimate uncertainty

**Why Bootstrap?**
- **Non-parametric**: No assumptions about the underlying distribution
- **Flexible**: Works for any statistic (mean, median, variance, correlation, etc.)
- **Practical**: Useful when theoretical formulas are unavailable or complex
- **Small samples**: Particularly valuable for small sample sizes

**Bootstrap Algorithm:**
1. Given original sample: $X_1, X_2, \ldots, X_n$
2. For $b = 1$ to $B$ (number of bootstrap samples):
   - Draw $n$ observations **with replacement** from the original sample: $X_1^{*b}, X_2^{*b}, \ldots, X_n^{*b}$
   - Compute the statistic: $\hat{\theta}^{*b} = g(X_1^{*b}, X_2^{*b}, \ldots, X_n^{*b})$
3. Use the distribution of $\{\hat{\theta}^{*1}, \hat{\theta}^{*2}, \ldots, \hat{\theta}^{*B}\}$ for inference

### 2.2 Bootstrap Standard Error

The **bootstrap standard error** estimates the variability of a statistic.

**Formula:**
$$\text{SE}_{boot}(\hat{\theta}) = \sqrt{\frac{1}{B-1}\sum_{b=1}^{B}(\hat{\theta}^{*b} - \bar{\theta}^*)^2}$$

where $\bar{\theta}^* = \frac{1}{B}\sum_{b=1}^{B}\hat{\theta}^{*b}$

**Interpretation:**
- This is the standard deviation of the bootstrap distribution
- It estimates how much $\hat{\theta}$ would vary if we repeated the experiment

In [ ]:
# Example: Bootstrap Standard Error for the Mean

# Original sample
np.random.seed(42)
original_sample = np.random.normal(loc=100, scale=15, size=50)

# Bootstrap parameters
B = 10000  # Number of bootstrap samples
n = len(original_sample)

# Perform bootstrap
bootstrap_means = []
for b in range(B):
    # Resample with replacement
    bootstrap_sample = np.random.choice(original_sample, size=n, replace=True)
    # Compute statistic (mean)
    bootstrap_means.append(np.mean(bootstrap_sample))

bootstrap_means = np.array(bootstrap_means)

# Bootstrap standard error
se_boot = np.std(bootstrap_means, ddof=1)

# Theoretical standard error (for comparison)
se_theory = np.std(original_sample, ddof=1) / np.sqrt(n)

print("Bootstrap Standard Error Estimation")
print("=" * 60)
print(f"Original sample size: n = {n}")
print(f"Original sample mean: {np.mean(original_sample):.4f}")
print(f"Original sample std: {np.std(original_sample, ddof=1):.4f}")
print(f"Number of bootstrap samples: B = {B}")
print(f"\nBootstrap standard error: {se_boot:.4f}")
print(f"Theoretical standard error: {se_theory:.4f}")
print(f"Difference: {abs(se_boot - se_theory):.4f}")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bootstrap distribution
axes[0].hist(bootstrap_means, bins=50, density=True, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].axvline(x=np.mean(original_sample), color='r', linestyle='--', linewidth=2, 
                label=f'Original mean: {np.mean(original_sample):.2f}')
axes[0].axvline(x=np.mean(bootstrap_means), color='g', linestyle='--', linewidth=2, 
                label=f'Bootstrap mean: {np.mean(bootstrap_means):.2f}')
# Add normal curve
x_range = np.linspace(min(bootstrap_means), max(bootstrap_means), 1000)
axes[0].plot(x_range, stats.norm.pdf(x_range, np.mean(bootstrap_means), se_boot), 
             'orange', linewidth=2, label='Normal approximation')
axes[0].set_xlabel('Bootstrap Mean', fontsize=12)
axes[0].set_ylabel('Density', fontsize=12)
axes[0].set_title(f'Bootstrap Distribution of Sample Mean\n(B={B} resamples)', 
                  fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Q-Q plot to check normality
stats.probplot(bootstrap_means, dist="norm", plot=axes[1])
axes[1].set_title('Q-Q Plot: Bootstrap Distribution vs Normal', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.3 Bootstrap Confidence Intervals

Bootstrap can be used to construct confidence intervals without assuming normality.

**Three Common Methods:**

1. **Percentile Method** (simplest):
   - $(\alpha/2)$ and $(1-\alpha/2)$ percentiles of bootstrap distribution
   - 95% CI: $[\hat{\theta}^*_{0.025}, \hat{\theta}^*_{0.975}]$

2. **Normal Method**:
   - Assumes normality: $\hat{\theta} \pm z_{\alpha/2} \cdot SE_{boot}$
   - 95% CI: $[\hat{\theta} - 1.96 \cdot SE_{boot}, \hat{\theta} + 1.96 \cdot SE_{boot}]$

3. **BCa Method** (Bias-Corrected and Accelerated):
   - Most accurate but more complex
   - Corrects for bias and skewness

In [ ]:
# Example: Bootstrap Confidence Intervals

# Using the bootstrap means from previous example
alpha = 0.05  # 95% confidence level

# Method 1: Percentile method
ci_percentile = np.percentile(bootstrap_means, [100*alpha/2, 100*(1-alpha/2)])

# Method 2: Normal method
z_critical = stats.norm.ppf(1 - alpha/2)
ci_normal = [np.mean(original_sample) - z_critical * se_boot,
             np.mean(original_sample) + z_critical * se_boot]

# Theoretical CI (for comparison)
ci_theory = stats.t.interval(1-alpha, df=n-1, loc=np.mean(original_sample), 
                              scale=se_theory)

print("Bootstrap Confidence Intervals (95%)")
print("=" * 60)
print(f"Point estimate: {np.mean(original_sample):.4f}")
print(f"\n1. Percentile Method:")
print(f"   [{ci_percentile[0]:.4f}, {ci_percentile[1]:.4f}]")
print(f"   Width: {ci_percentile[1] - ci_percentile[0]:.4f}")
print(f"\n2. Normal Method:")
print(f"   [{ci_normal[0]:.4f}, {ci_normal[1]:.4f}]")
print(f"   Width: {ci_normal[1] - ci_normal[0]:.4f}")
print(f"\n3. Theoretical t-interval (for comparison):")
print(f"   [{ci_theory[0]:.4f}, {ci_theory[1]:.4f}]")
print(f"   Width: {ci_theory[1] - ci_theory[0]:.4f}")

# Visualize
plt.figure(figsize=(14, 7))
plt.hist(bootstrap_means, bins=50, density=True, alpha=0.5, color='steelblue', edgecolor='black', label='Bootstrap distribution')

# Mark the confidence intervals
plt.axvline(x=ci_percentile[0], color='red', linestyle='--', linewidth=2, label='Percentile CI')
plt.axvline(x=ci_percentile[1], color='red', linestyle='--', linewidth=2)
plt.axvspan(ci_percentile[0], ci_percentile[1], alpha=0.2, color='red')

plt.axvline(x=ci_normal[0], color='green', linestyle=':', linewidth=2, label='Normal CI')
plt.axvline(x=ci_normal[1], color='green', linestyle=':', linewidth=2)

plt.axvline(x=np.mean(original_sample), color='blue', linestyle='-', linewidth=3, 
            label=f'Sample mean: {np.mean(original_sample):.2f}')

plt.xlabel('Bootstrap Mean', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.title('Bootstrap Confidence Intervals Comparison', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 2.4 Bootstrap for Other Statistics

Bootstrap is particularly useful for statistics where theoretical formulas are complex or unavailable.

**Examples:**
- Median
- Correlation coefficient
- Ratio of means
- Regression coefficients
- Machine learning model performance metrics

In [ ]:
# Example: Bootstrap for Median

# Generate skewed data (median is more robust than mean for skewed data)
np.random.seed(123)
skewed_data = np.random.exponential(scale=10, size=60)

# Bootstrap for median
B = 10000
n = len(skewed_data)
bootstrap_medians = []

for b in range(B):
    bootstrap_sample = np.random.choice(skewed_data, size=n, replace=True)
    bootstrap_medians.append(np.median(bootstrap_sample))

bootstrap_medians = np.array(bootstrap_medians)

# Bootstrap standard error and CI
se_median = np.std(bootstrap_medians, ddof=1)
ci_median = np.percentile(bootstrap_medians, [2.5, 97.5])

print("Bootstrap for Median (Skewed Data)")
print("=" * 60)
print(f"Sample size: n = {n}")
print(f"Sample mean: {np.mean(skewed_data):.4f}")
print(f"Sample median: {np.median(skewed_data):.4f}")
print(f"\nBootstrap results:")
print(f"  Bootstrap SE(median): {se_median:.4f}")
print(f"  95% CI for median: [{ci_median[0]:.4f}, {ci_median[1]:.4f}]")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Original data
axes[0].hist(skewed_data, bins=30, alpha=0.7, color='coral', edgecolor='black')
axes[0].axvline(x=np.mean(skewed_data), color='blue', linestyle='--', linewidth=2, 
                label=f'Mean: {np.mean(skewed_data):.2f}')
axes[0].axvline(x=np.median(skewed_data), color='green', linestyle='--', linewidth=2, 
                label=f'Median: {np.median(skewed_data):.2f}')
axes[0].set_xlabel('Value', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Original Skewed Data', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')

# Bootstrap distribution of median
axes[1].hist(bootstrap_medians, bins=50, density=True, alpha=0.7, color='lightgreen', edgecolor='black')
axes[1].axvline(x=np.median(skewed_data), color='red', linestyle='--', linewidth=2, 
                label=f'Sample median: {np.median(skewed_data):.2f}')
axes[1].axvline(x=ci_median[0], color='orange', linestyle=':', linewidth=2, label='95% CI')
axes[1].axvline(x=ci_median[1], color='orange', linestyle=':', linewidth=2)
axes[1].axvspan(ci_median[0], ci_median[1], alpha=0.2, color='orange')
axes[1].set_xlabel('Bootstrap Median', fontsize=12)
axes[1].set_ylabel('Density', fontsize=12)
axes[1].set_title(f'Bootstrap Distribution of Median (B={B})', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Example: Bootstrap for Correlation Coefficient

# Generate correlated data
np.random.seed(456)
n = 50
x = np.random.normal(0, 1, n)
y = 0.7 * x + np.random.normal(0, 0.5, n)  # Correlation ≈ 0.7

# Original correlation
corr_original = np.corrcoef(x, y)[0, 1]

# Bootstrap for correlation
B = 10000
bootstrap_corrs = []

for b in range(B):
    # Resample pairs (important!)
    indices = np.random.choice(n, size=n, replace=True)
    x_boot = x[indices]
    y_boot = y[indices]
    bootstrap_corrs.append(np.corrcoef(x_boot, y_boot)[0, 1])

bootstrap_corrs = np.array(bootstrap_corrs)

# Bootstrap SE and CI
se_corr = np.std(bootstrap_corrs, ddof=1)
ci_corr = np.percentile(bootstrap_corrs, [2.5, 97.5])

print("Bootstrap for Correlation Coefficient")
print("=" * 60)
print(f"Sample size: n = {n}")
print(f"Sample correlation: r = {corr_original:.4f}")
print(f"\nBootstrap results:")
print(f"  Bootstrap SE(r): {se_corr:.4f}")
print(f"  95% CI for r: [{ci_corr[0]:.4f}, {ci_corr[1]:.4f}]")

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Scatter plot
axes[0].scatter(x, y, alpha=0.6, color='steelblue', edgecolor='black')
# Add regression line
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
axes[0].plot(x, p(x), "r--", linewidth=2, label=f'r = {corr_original:.3f}')
axes[0].set_xlabel('X', fontsize=12)
axes[0].set_ylabel('Y', fontsize=12)
axes[0].set_title('Original Data: X vs Y', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Bootstrap distribution of correlation
axes[1].hist(bootstrap_corrs, bins=50, density=True, alpha=0.7, color='lightcoral', edgecolor='black')
axes[1].axvline(x=corr_original, color='red', linestyle='--', linewidth=2, 
                label=f'Sample r: {corr_original:.3f}')
axes[1].axvline(x=ci_corr[0], color='orange', linestyle=':', linewidth=2, label='95% CI')
axes[1].axvline(x=ci_corr[1], color='orange', linestyle=':', linewidth=2)
axes[1].axvspan(ci_corr[0], ci_corr[1], alpha=0.2, color='orange')
axes[1].set_xlabel('Bootstrap Correlation', fontsize=12)
axes[1].set_ylabel('Density', fontsize=12)
axes[1].set_title(f'Bootstrap Distribution of Correlation (B={B})', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.5 Comparing Bootstrap with Theoretical Results

Let's compare bootstrap with theoretical results for a case where we know the theory.

In [ ]:
# Comparison: Bootstrap vs Theory for Sample Mean

# Simulation parameters
true_mu = 50
true_sigma = 10
sample_size = 30
n_simulations = 1000
B = 5000

# Storage
coverage_theory = 0
coverage_bootstrap = 0
widths_theory = []
widths_bootstrap = []

np.random.seed(789)

for sim in range(n_simulations):
    # Generate sample
    sample = np.random.normal(true_mu, true_sigma, sample_size)
    
    # Theoretical CI
    se_theory = np.std(sample, ddof=1) / np.sqrt(sample_size)
    ci_theory = stats.t.interval(0.95, df=sample_size-1, loc=np.mean(sample), scale=se_theory)
    widths_theory.append(ci_theory[1] - ci_theory[0])
    if ci_theory[0] <= true_mu <= ci_theory[1]:
        coverage_theory += 1
    
    # Bootstrap CI
    bootstrap_means = []
    for b in range(B):
        boot_sample = np.random.choice(sample, size=sample_size, replace=True)
        bootstrap_means.append(np.mean(boot_sample))
    ci_boot = np.percentile(bootstrap_means, [2.5, 97.5])
    widths_bootstrap.append(ci_boot[1] - ci_boot[0])
    if ci_boot[0] <= true_mu <= ci_boot[1]:
        coverage_bootstrap += 1

# Results
print("Bootstrap vs Theoretical Comparison")
print("=" * 60)
print(f"True parameters: μ = {true_mu}, σ = {true_sigma}")
print(f"Sample size: n = {sample_size}")
print(f"Number of simulations: {n_simulations}")
print(f"Bootstrap resamples per simulation: B = {B}")
print(f"\nCoverage (should be ≈95%):")
print(f"  Theoretical t-interval: {coverage_theory/n_simulations*100:.2f}%")
print(f"  Bootstrap percentile: {coverage_bootstrap/n_simulations*100:.2f}%")
print(f"\nAverage CI width:")
print(f"  Theoretical: {np.mean(widths_theory):.4f}")
print(f"  Bootstrap: {np.mean(widths_bootstrap):.4f}")

# Visualize
plt.figure(figsize=(14, 7))
plt.hist(widths_theory, bins=30, alpha=0.5, color='blue', edgecolor='black', label='Theoretical CI widths')
plt.hist(widths_bootstrap, bins=30, alpha=0.5, color='red', edgecolor='black', label='Bootstrap CI widths')
plt.axvline(x=np.mean(widths_theory), color='blue', linestyle='--', linewidth=2, 
            label=f'Mean (Theory): {np.mean(widths_theory):.2f}')
plt.axvline(x=np.mean(widths_bootstrap), color='red', linestyle='--', linewidth=2, 
            label=f'Mean (Bootstrap): {np.mean(widths_bootstrap):.2f}')
plt.xlabel('CI Width', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title(f'Confidence Interval Widths: Bootstrap vs Theory\n({n_simulations} simulations)', 
          fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## Part 3: Comparison and Summary

### 3.1 Method of Moments vs Bootstrap

| Aspect | Method of Moments | Bootstrap |
|--------|-------------------|----------|
| **Type** | Parametric | Non-parametric |
| **Assumptions** | Requires distributional form | No distributional assumptions |
| **Purpose** | Point estimation of parameters | Uncertainty quantification (SE, CI) |
| **Computation** | Analytical (usually simple) | Computational (intensive) |
| **Sample Size** | Works for any size | Better with larger samples |
| **Flexibility** | Limited to moments | Works for any statistic |
| **Efficiency** | May not be optimal | Approaches optimal with large B |

### 3.2 When to Use Each Method

**Use Method of Moments when:**
- You know the distributional family (e.g., Normal, Exponential)
- You need quick point estimates
- Moments are easy to compute
- MLE is difficult or intractable

**Use Bootstrap when:**
- You need standard errors or confidence intervals
- The statistic is complex (median, correlation, etc.)
- You don't want to assume a distribution
- Theoretical formulas are unavailable
- Sample size is small to moderate

### 3.3 Key Takeaways

1. **Method of Moments** provides simple parameter estimates by matching sample and theoretical moments
2. **Bootstrap** is a powerful resampling technique for estimating uncertainty without distributional assumptions
3. Both methods are complementary: MoM for point estimation, Bootstrap for inference
4. Bootstrap is particularly valuable in machine learning for model evaluation and uncertainty quantification
5. Always check assumptions and validate results when possible

## Part 4: Practice Exercises

Try these exercises to reinforce your understanding!

In [ ]:
# Exercise 1: Method of Moments for Uniform Distribution
# Given data from Uniform(0, θ), estimate θ using MoM
# Hint: E[X] = θ/2, so θ̂ = 2*mean(X)

# Generate data
theta_true = 10
data_uniform = np.random.uniform(0, theta_true, size=100)

# Your code here:
# theta_mom = ...

print("Exercise 1: Uniform Distribution MoM")
print(f"True θ: {theta_true}")
# print(f"MoM estimate: {theta_mom:.4f}")

In [ ]:
# Exercise 2: Bootstrap for Standard Deviation
# Compute bootstrap SE and 95% CI for the standard deviation

# Generate data
data_ex2 = np.random.normal(100, 15, size=50)

# Your code here:
# B = 10000
# bootstrap_stds = []
# for b in range(B):
#     ...

print("Exercise 2: Bootstrap for Standard Deviation")
print(f"Sample std: {np.std(data_ex2, ddof=1):.4f}")
# print(f"Bootstrap SE: ...")
# print(f"95% CI: [...]")

In [ ]:
# Exercise 3: Compare MoM and MLE for Exponential Distribution
# Compute both estimates and compare
# MLE for Exponential: λ̂_MLE = n / Σxᵢ (same as MoM!)

# Generate data
lambda_true_ex3 = 1.5
data_ex3 = np.random.exponential(scale=1/lambda_true_ex3, size=200)

# Your code here:
# lambda_mom = ...
# lambda_mle = ...

print("Exercise 3: MoM vs MLE for Exponential")
print(f"True λ: {lambda_true_ex3}")
# print(f"MoM: {lambda_mom:.4f}")
# print(f"MLE: {lambda_mle:.4f}")
# print(f"Are they the same? {np.isclose(lambda_mom, lambda_mle)}")